Mount the google drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Building the training datasets

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import tensorflow as tf
import numpy as np # linear algebra
import os
import sys
import time
import gc
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import datetime

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

sys.path.append('/content/drive/MyDrive/AdaVFL-GitHub')
from Data import MakeTrainingTimes, CheckLocalTrainData, CheckLocalPredictionData, GenerateRandomSamples, LocalSequentialDataset, LocalSampledDataset, GlobalSequentialDataset, GlobalSampledDataset, build_adjMatrix, MakeAttackTimes
from federationarguments import arguments

args = arguments()
# args = arguments("Yelp",8,8)
if(args.dataset == "bikeNYC"):
  link = "https://drive.google.com/drive/folders/1diJwebRNa5AQ16Jy6eHNGtYGmIGeqcrt"
  args.trainingInterval = 24*60*60*1000
  args.x_axis = 8
  args.y_axis = 16
else:
  if(args.dataset == "Yelp"):
    link = "https://drive.google.com/drive/folders/1K2Y_txKAda0TOEEDYvoa7sPPMYLCXI-U"
    args.trainingInterval = 8*24*60*60*1000
    args.x_axis = 8
    args.y_axis = 8
  else:
    raise SystemError('Invalid data folder') 

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
  args.device_name = 'cuda'
  torch.cuda.set_device(0)
else:
  args.device_name = 'cpu'
  # raise SystemError('GPU device not found')

fluff, folder = link.split('folders/')
#print (id) 
filePath = "'%s' in parents and trashed=false" %folder
print(filePath)
downloaded = drive.ListFile({'q':filePath}).GetList()

LocalTrainData = []
LocalValidationData = []
LocalTestData = []
LocalPredictionSamples = {}
GlobalPredictionSamples = GlobalSampledDataset(args)
GlobalTestData = GlobalSequentialDataset(args,args.testRatioBegin,args.testRatioEnd)
AttackerTrainData = GlobalSequentialDataset(args,args.trainAttackerBegin,args.trainAttackerEnd)
attacker_test_dataset = GlobalSampledDataset(args)

sampled_list = []
sampledPrediction = True
if len(downloaded) > 0:
  sampledPrediction = True
  for file in downloaded:
    try:
      downloaded = drive.CreateFile({'id':file['id']}) 
      downloaded.GetContentFile(file['title'])  
      timestamps = pd.read_csv(file['title'])
      grid = file['title'].split(".")
      axis = grid[0].split("X")
      x_axis = int(axis[0])
      y_axis = int(axis[1])
      train = LocalSequentialDataset(timestamps,x_axis, y_axis, args.trainRatioBegin,args.trainRatioEnd, args)
      train.make_data()
      LocalTrainData.append(train)

      if(sampledPrediction == True):
          sampled_list = GenerateRandomSamples(timestamps, args)    
          sampledPrediction = False  

      predictsample = LocalSampledDataset(timestamps, x_axis, y_axis, args)
      predictsample.make_data(sampled_list)
      LocalValidationData.append(predictsample)
      LocalValidationData.append(predictsample)
      test = LocalSequentialDataset(timestamps,x_axis, y_axis, args.testRatioBegin,args.testRatioEnd, args)
      test.make_data()
      LocalTestData.append(test)
      sample_ID = str(x_axis)+"X"+str(y_axis)
      LocalPredictionSamples[sample_ID] = predictsample
      GlobalTestData.add_data(file['title'], timestamps)
      GlobalPredictionSamples.add_data(file['title'], predictsample)
      AttackerTrainData.add_data(file['title'], timestamps)
      attacker_test_dataset.add_data(file['title'], predictsample)
    except Exception as e:
        print("hit an exception when making data ",e)
        exit('hit an exception when making data')    

GlobalTestData.make_data()
GlobalTestData.check_data()

AttackerTrainData.make_data()
AttackerTrainData.check_data()

MakeAttackTimes(AttackerTrainData,args)
MakeTrainingTimes(LocalTrainData, args)

CheckLocalTrainData(LocalTrainData, args)

CheckLocalPredictionData(LocalPredictionSamples, sampled_list)

GlobalPredictionSamples.make_data()
GlobalPredictionSamples.check_data()

attacker_test_dataset.make_data()
attacker_test_dataset.check_data()
print("len(LocalPredictionSamples)",len(LocalPredictionSamples))

adj = build_adjMatrix(args)
beginningTime = args.beginTrainingTimestamp
endingTime = args.endTrainingTimestamp
iterations = args.epochs*(((endingTime - beginningTime)//args.trainingInterval)+1)

print("starting training time", datetime.datetime.fromtimestamp(args.start_training_time/1000).strftime('%Y-%m-%d %H:%M:%S'))
print("ending training time ", datetime.datetime.fromtimestamp(args.start_ending_time/1000).strftime('%Y-%m-%d %H:%M:%S'))

del sampled_list
del timestamps
gc.collect()


'1K2Y_txKAda0TOEEDYvoa7sPPMYLCXI-U' in parents and trashed=false
GlobalSequentialDataset:size of seqDict 434
GlobalSequentialDataset:size of labelDict 434
GlobalSequentialDataset min value tensor(-1.)
GlobalSequentialDataset max value tensor(-1.)
start timestamp of global sequential 1599908399000
end timestamp of global sequential 1609433999000
GlobalSequentialDataset:size of seqDict 868
GlobalSequentialDataset:size of labelDict 868
GlobalSequentialDataset min value tensor(-1.)
GlobalSequentialDataset max value tensor(-1.)
start timestamp of global sequential 1562108399000
end timestamp of global sequential 1581008399000
CheckLocalTrainData
beginningTime= 1514782799000
endingTime= 1538197199000
args.trainingInterval= 691200000
CheckLocalPredictionData
size of seqDict 657
GlobalSampledDataset
GlobalSampledDataset:size of seqDict 657
GlobalSampledDataset min value -1.0
GlobalSampledDataset max value 1.0
GlobalSampledDataset
GlobalSampledDataset:size of seqDict 657
GlobalSampledDataset mi

791

Run the privacy attack

In [ ]:
import os
from Models import GRU, MyGAT
from tqdm import tqdm
from Data import build_map,build_adjMatrix
from WeightTools import train_global_weights, test_model
import random 
from Data import compute_feature_contribution,initialize_ratio_contribution 
from Initial_feature_weights import initialize_avg_computed_features
from Models import MLP
from tqdm import tqdm
from Data import modify_input_attacker, build_attacker_input, extract_true_input, initialize_ratio_outputsize, train_attacker
import datetime
import numpy as np
import random
from torch.optim.lr_scheduler import ExponentialLR

###########################
# upload the trained global model
adj = adj.to(args.device_name)
global_test_output=[]

globalModel = None
if (args.global_model == 'GNN'):
  globalModel = MyGAT(args,adj)
  globalModel.to(args.device_name)
else:
  exit('Error: unrecognized global model')

print("The feature mode to be tested ",args.featureMode)
ratio = args.featureRatio

if args.dataset == "Yelp":
  directory_name = "/content/drive/MyDrive/Colab Notebooks/Models/Yelp/"
else:
  directory_name = "/content/drive/MyDrive/Colab Notebooks/Models/bikenyc/"

if args.featureMode== "None":
  file_name = "NoPrivacyGAN.pth"
else:
  if args.featureMode== "Uniform":
    file_name = "UniformPrivacyGAN.pth"
  else:
    if args.featureMode== "InvContribution":
      file_name = 'InvConPrivacyGAN{}.pth'.format(ratio)
    else:
      if args.featureMode== "LinContribution":
        file_name = 'LinConPrivacyGAN{}.pth'.format(ratio)
        
print("The target model's to upload ",file_name)

path = '{}{}'.format(directory_name,file_name)
map_location='cpu'
globalModel.load_state_dict(torch.load(path,map_location=map_location))
globalModel.eval()

########################### 
#initialize the attacker's model
feature_contribution_metrics = initialize_avg_computed_features(args)
organization_contribution_distribution = initialize_ratio_contribution(args,feature_contribution_metrics)
feature_contribution_metrics = compute_feature_contribution(organization_contribution_distribution,feature_contribution_metrics,args)
attacker = MLP(args,initialize_ratio_outputsize(args))
attacker.to(args.device_name)
attacker.train()

###########################
#initialize variables
beginningTime = args.beginAttackTimestamp
endingTime = args.endAttackTimestamp-args.trainingInterval
iterations = ((endingTime - beginningTime)//args.trainingInterval)+1

batch_loss = []
batch_MAE = []
batch_AE = []
batch_RMSE = []
batch_WMAPE = []

###########################
#train the attacker's model
organization_contribution_distribution = np.tile(organization_contribution_distribution,(args.input_length,1,1))  
flat_organization_contribution_distribution = np.copy(organization_contribution_distribution)
flat_organization_contribution_distribution = np.ndarray.flatten(flat_organization_contribution_distribution)
for epoch in tqdm(range(args.attacker_epochs)):
#     for epoch in tqdm(range(10)):
# while loss > 0.19:
#       for i in tqdm(range(iterations)):
  for i in range(iterations):
    timestamp = random.randrange(beginningTime, endingTime, args.trainingInterval)
    accuracy_training, loss = train_attacker(attacker,timestamp,AttackerTrainData,globalModel,adj,organization_contribution_distribution, args)
#         print("batch loss",loss)
    batch_loss.append(loss)
    batch_MAE.append(accuracy_training["MAE"])
    batch_RMSE.append(accuracy_training["RMSE"])
    batch_AE.append(accuracy_training["AE"])
    batch_WMAPE.append(accuracy_training["WMAPE"])
#         print("batch accuracy_training[AE]",accuracy_training["AE"])
#         print("batch accuracy_training[RMSE]",accuracy_training["RMSE"])

###########################
#test the attacker model and output metrics
attacker.eval()
acc = {}
acc["MSE"] = 0.
acc["MAE"] = 0.
acc["RMSE"] = 0.
with torch.no_grad():
  predictions = []  
  attacker_accuracy = []
#       MSE_accuracy = 0.      
  sequence_number = 0
  for seq, labels in attacker_test_dataset.sequence:
    accuracy, predictions= attacker.guess(seq,flat_organization_contribution_distribution,args.attacker_organization_id)
    sequence_number += 1
    acc["MSE"] += accuracy["MSE"]  
    acc["RMSE"] += np.sqrt(accuracy["MSE"]) 
    acc["MAE"] += accuracy["MAE"] 
  acc["MSE"] /= sequence_number
  acc["RMSE"] /= sequence_number
  acc["MAE"] /= sequence_number
  print("the attacker success MSE=", acc["MSE"], ",RMSE=",acc["RMSE"],",MAE=",acc["MAE"])

The feature mode to be tested  LinContribution
The target model's to upload  LinConPrivacyGAN90.pth
initialize_avg_computed_features: Yelp


100%|██████████| 6/6 [18:30<00:00, 185.09s/it]


the attacker success MSE= tensor(1.2575) ,RMSE= tensor(1.1208) ,MAE= tensor(-0.5424)
